# Climbing logs: exploration

Exploration of the usability of the logs

### 0. Variables and dependencies

In [ ]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from climb.conversion import convert_usa2french
from climb.extraction import extract_climbinglogs, extract_sessionlogs

In [ ]:
filein = './../data/climbinglogs.xlsx'

In [ ]:
# Temporary hotfix

import json
from pandas.api.types import CategoricalDtype

# Beware: these are currently ordered
# Yet, it is not guaranteed !!!
with open('./grades.json') as ff:
    grades = json.load(ff)

grades = list(grades['French2USA'].keys())
grades = CategoricalDtype(categories=grades,
                          ordered=True)

In [ ]:
# Temporary hotfix
ascensions = ['flash', 'redpoint', 'repeat', 'topped', 'not topped']
ascensions = CategoricalDtype(categories=ascensions,
                              ordered=True)

### 1. Read in the data

In [ ]:
logs = extract_climbinglogs(filein,
                            cols_ffill=['date', 'climbing_gym', 'difficulty_level']
                           )
# Rename columns
logs = logs.rename(columns={'difficulty_level': 'grade'})

# Fill missing values
logs.loc[:, ['blocks', 'falls', 'sends']] = logs.loc[:,['blocks', 'falls', 'sends']].fillna(0)

for cc in ['blocks', 'falls', 'sends']:
    logs[cc] = pd.to_numeric(logs[cc], downcast='integer')

# Move certain values to lowercase
for cc in ['ascension_type', 'style']:
    logs[cc] = [val.lower() for val in logs[cc]]
    
# Convert to font grade system
logs['grade'] = [convert_usa2french(val) for val in logs['grade']]

# Create catergorical data
logs['grade'] = logs['grade'].astype(grades)
logs['ascension_type'] = logs['ascension_type'].astype(ascensions)

In [ ]:
logs

In [ ]:
#logs['grade'].median()

### 2. Quick pyramid plot

Was actually the result of some iterative developing

In [ ]:
def create_pyramid(df, aggtype: str = 'sum'):
    """
    Create the pyramid aggregation of the climbing logs
    """
    # The actual pyramid
    pyrm = (df
            .groupby(['grade', 'ascension_type'])
            .agg(sends=('sends', aggtype),
                )
           )
    
    # Additional cumulative sum, needed to display the pyramid
    temp = pyrm.groupby(level=0).cumsum()
    temp = temp.rename(columns={'sends': 'sends_cumsum'})
    
    # Join the two dataframes
    pyrm = pyrm.reset_index()
    temp = temp.reset_index()
    
    return pyrm.merge(temp, on=['grade', 'ascension_type'], how='left')  

In [ ]:
# Needs to be generalized to accept axis!
def plot_pyramid(pyrm: pd.DataFrame, ax=None, legend=False, gradesystem='French'):
    """
    Plot the pyramid diagram
    """
    assert 'grade' in pyrm.columns
    assert 'ascension_type' in pyrm.columns
    assert 'sends' in pyrm.columns
    assert 'sends_cumsum' in pyrm.columns
    
    # Bookkeeping
    category_colors = plt.get_cmap('viridis')(np.linspace(0.0, 1., len(ascensions.categories)))
    labels = pyrm['grade']
    
    # Create a figure in case it was not specified
    if ax == None:
        fig = plt.figure(figsize=(16,16))
        ax = fig.add_subplot(111)
    
    
    
    # Draw the pyramid
    for aa, color in zip(ascensions.categories, category_colors):
        # Subset the pyramid data to one ascension type
        data = pyrm[pyrm['ascension_type'] == aa]
        
        widths = data['sends']
        starts = data['sends_cumsum'] - widths
        
        # Actual plotting
        ax.barh(data['grade'], widths, left=starts, height=0.85,
                label=aa, color=color, edgecolor='k')
        
        xcenters = starts + widths / 2
        
        # Get the colors our to determine annotation color
        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        for y, (x, c) in enumerate(zip(xcenters, widths)):
            if c != int(0):
                ax.text(x, y, str(int(c)), ha='center', va='center',
                        color=text_color)
    
    # Make the figure meaningful   
    if legend:
        ax.legend(ncol=len(ascensions.categories),
                  bbox_to_anchor=(0, 1),
                  loc='lower left', fontsize='large')
    
    ax.set_ylim(['4', '7c'])
    ax.set_xlabel('Total routes climbed', fontsize='large')
    ax.set_ylabel('Climbing grade (Font system)', fontsize='large')
    
    ax.set_xlim([0, pyrm['sends_cumsum'].max() + 2])
    
    if ax == None:
        return fig, ax
    else:
        return ax

In [ ]:
condlead = logs['style'] == 'lead'
condtop = logs['style'] == 'toprope'

fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
fig.subplots_adjust(wspace=0)

pyrmlead = create_pyramid(logs[condlead])
pyrmtop = create_pyramid(logs[condtop])
ax1 = plot_pyramid(pyrmlead, ax1)
ax2 = plot_pyramid(pyrmtop, ax2)

ax1.invert_xaxis()
ax1.set_title('Lead climbing - {} routes'.format(pyrmlead['sends'].sum()))
ax2.set_title('Toprope climbing - {} routes'.format(pyrmtop['sends'].sum()))
ax2.yaxis.set_label_position('right')
ax2.yaxis.tick_right()
ax2.legend(ncol=len(ascensions.categories),
           bbox_to_anchor=(0, -0.12),
           loc='best', fontsize='large')

xmax = max([ax1.get_xlim()[1], ax2.get_xlim()[1]])
ax1.set_xlim([xmax, 0]); ax2.set_xlim([0, xmax]);
